<a href="https://colab.research.google.com/github/gulyaaa18/gulyaaa/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_2_%D0%90%D1%81%D0%B3%D0%B0%D1%80%D0%BB%D0%B8_%D0%93%D1%83%D0%BB%D0%B0%D1%80_%D0%9A%D0%9C%D0%91%D0%9E_07_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext sql
%sql sqlite:///store.db


In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///store.db
Done.


name
albums
sqlite_sequence
artists
genres
invoice_items
media_types
playlists
playlist_track
tracks
invoices


## Задание 1
Вывести 5 самых больших по размеру (в байтах) треков жанра Blues и формата MPEG. Только название и размер.


In [ ]:
%%sql
SELECT tracks.Name, Bytes FROM tracks
JOIN  genres ON genres.genreId = tracks.genreid
WHERE genres.name = 'Blues' AND MediaTypeId = 3
ORDER BY Bytes DESC
LIMIT 5;

 * sqlite:///store.db
Done.


Name,Bytes
Old Love,15780747
Running On Faith,12536275
Lonely Stranger,10894406
Layla,9490542
Tears In Heaven,9032835


## Задание 2
Вывести названия всех треков и их исполнителей для всех треков жанра Rock, приобретенных сотрудниками Oracle.


In [ ]:
%%sql
SELECT tracks.Name AS trackName, artists.Name as ArtistName -- , customers.CustomerId AS CusId
FROM tracks
-- Сначала вытащим исполнителя
JOIN albums ON tracks.AlbumId = albums.AlbumId -- Объединяем айдишники альбомов
JOIN artists ON albums.ArtistId = artists.ArtistId -- Объединяем айдишники с исполнителями
-- Теперь вытащим всех сотрудников Oracle
JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId
JOIN invoices ON invoice_items.InvoiceId = invoices.InvoiceId
JOIN customers ON invoices.CustomerId = customers.CustomerId
JOIN  genres ON genres.genreId = tracks.genreid
WHERE genres.name = 'Rock'  AND
customers.Company = 'Oracle'

 * sqlite:///store.db
Done.


trackName,ArtistName
Sunday Bloody Sunday,U2
New Year's Day,U2
That's The Way,Led Zeppelin
Ten Years Gone,Led Zeppelin
Achilles Last Stand,Led Zeppelin
Tea For One,Led Zeppelin
No Quarter,Led Zeppelin
Bad Moon Rising,Creedence Clearwater Revival
Down On The Corner,Creedence Clearwater Revival
Up Around The Bend,Creedence Clearwater Revival


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций (если 0, можно не выводить).


In [ ]:
%%sql
-- COUNT(invoices.CustomerId) - Подсчет всех счетов
SELECT Company, count(invoice_items.quantity) AS JazzCount FROM customers
JOIN invoices ON invoices.CustomerId = customers.CustomerId -- Объединяем айди пользователей в таблицах invoices и customers
JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId -- Объединяем айди счета в таблицах invoices и invoice_items
JOIN tracks ON tracks.TrackId = invoice_items.TrackId -- Объединяем айди треков в таблицах tracks и invoice_items
JOIN  genres ON genres.genreId = tracks.genreid
-- Где поле компания не пустое и где жанр трека Джаз
WHERE Company IS NOT NULL AND genres.name = 'Jazz'
GROUP BY Company

 * sqlite:///store.db
Done.


Company,JazzCount
Apple Inc.,2
BMW,1
Google Inc.,12
JetBrains s.r.o.,3
Microsoft Corporation,15
Oracle,3
SAP,1
Telus,2


## Задание 4
Вывести топ 3 плейлиста по продолжительности (продолжительность плейлиста = сумма длин треков в нём). Только название и продолжительность (в часах).


In [ ]:
%%sql
SELECT  playlists.Name AS PlaylistName, SUM(tracks.Milliseconds)/1000/60/60 AS Duration_hours FROM playlist_track
JOIN tracks ON tracks.TrackId = playlist_track.TrackId
JOIN playlists ON playlist_track.playlistId = playlists.playlistId
GROUP BY PlaylistName
ORDER BY Duration_hours DESC
LIMIT 3

 * sqlite:///store.db
Done.


PlaylistName,Duration_hours
Hidden Gems,138
From That TV Show,138
90’s Music,10


## Задание 5
Для каждого набора (жанр, тип медиа) вывести количество треков в них, причем вывести только те наборы, для которых **все** треки стоят больше 1$ и для которых в наборе есть хотя бы один трек.

In [ ]:
%%sql
SELECT g.Name AS Genre, m.Name AS MediaType, COUNT(t.Name) AS COUNT
FROM tracks t
JOIN genres g, media_types m ON t.GenreId = g.GenreId AND t.mediaTypeId = m.mediaTypeId
GROUP BY g.Name, m.Name HAVING COUNT(t.TrackId) > 0 AND MIN(t.UnitPrice) > 1

 * sqlite:///store.db
Done.


Genre,MediaType,COUNT
Blues,AAC audio file,9
Comedy,MPEG audio file,17
Sci Fi & Fantasy,MPEG audio file,26
Science Fiction,MPEG audio file,13
TV Shows,MPEG audio file,93
